![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

In [94]:
df = pd.read_csv(R"C:\Users\PC\Desktop\Projects\DataCamp\workspace\workspace\bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64
mortgage
--------------
mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64
previous_outcome
--------------
previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64
campaign_outcome
--------------
campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64


In [95]:
df.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')

In [96]:
df.dtypes

client_id                       int64
age                             int64
job                            object
marital                        object
education                      object
credit_default                 object
mortgage                       object
month                          object
day                             int64
contact_duration                int64
number_contacts                 int64
previous_campaign_contacts      int64
previous_outcome               object
cons_price_idx                float64
euribor_three_months          float64
campaign_outcome               object
dtype: object

In [97]:
df.isna().sum()

client_id                     0
age                           0
job                           0
marital                       0
education                     0
credit_default                0
mortgage                      0
month                         0
day                           0
contact_duration              0
number_contacts               0
previous_campaign_contacts    0
previous_outcome              0
cons_price_idx                0
euribor_three_months          0
campaign_outcome              0
dtype: int64

  `Preprocessing For CLIENT` 

In [98]:
def client_cleaning(df):
    """"
        Check null values 
        Client's type of job | Change `"."` to `"_"` 
        Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN`
        Whether the client's credit is in default  Convert to `boolean` data type: `1` if `"yes"`, otherwise `0`
        Whether the client has an existing mortgage (housing loan) | Convert to boolean data type: `1` if `"yes"`, otherwise `0`

        param df : dataframe
        
        return : cleaned dataframe (for client)
    
    """
    
    df['job'] = df['job'].str.replace('.', '_')

    df['education'] = df['education'].str.replace('.', '_')
    df['education'] = np.where(df['education'].str.contains('unknown' , na=False) , np.nan , df['education'])
    
    df['credit_default'] = np.where(df['credit_default'].values == 'yes' ,1 ,0)
    
    df['mortgage'] =np.where(df['mortgage'].values == 'yes' ,1 ,0) 

In [99]:
client_cleaning(df)

In [100]:
df.dtypes

client_id                       int64
age                             int64
job                            object
marital                        object
education                      object
credit_default                  int64
mortgage                        int64
month                          object
day                             int64
contact_duration                int64
number_contacts                 int64
previous_campaign_contacts      int64
previous_outcome               object
cons_price_idx                float64
euribor_three_months          float64
campaign_outcome               object
dtype: object

  `Preprocessing For CAMPAIGN` 

In [116]:
def campaign_cleaning(df):
    
    """
        Check null values
        Convert to boolean data type:`1` if `"success"`, otherwise `0`.
        Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`
        Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`) **Format =** `"YYYY-MM-DD"` 
    
        param : df = DataFrame
        
        
    """
    
    df['previous_outcome'] = np.where(df['previous_outcome'].values == 'success' ,1 ,0)
    
    df['campaign_outcome'] = np.where(df['campaign_outcome'].values == 'yes' ,1 ,0)
    
    df['month'] = pd.to_datetime(df['month'], format='%b' ,errors='coerce').dt.month
    df['day'] = pd.to_numeric(df['day'] , errors='coerce')
    df['year'] = 2022
    df['last_contact_date'] = pd.to_datetime(df[['year', 'month', 'day']] , format='%Y-%m-%d')
    
campaign_cleaning(df)

In [117]:
client_columns= ['client_id' , 'age' ,'job','marital', 'education', 'credit_default','mortgage']
campaign_columns= ['client_id' , 'contact_duration', 'number_contacts','previous_campaign_contacts', 'previous_outcome','campaign_outcome','last_contact_date']
economics_columns = ['client_id','cons_price_idx','euribor_three_months']

In [118]:
client = df[client_columns]
campaign = df[campaign_columns]
economics = df[economics_columns]

In [119]:
client

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,0,0
1,1,57,services,married,high_school,0,0
2,2,37,services,married,high_school,0,1
3,3,40,admin_,married,basic_6y,0,0
4,4,56,services,married,high_school,0,0
...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,0,1
41184,41184,46,blue-collar,married,professional_course,0,0
41185,41185,56,retired,married,university_degree,0,1
41186,41186,44,technician,married,professional_course,0,0


In [120]:
campaign

,client_id,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,261,1,0,0,0,NaT
1,1,149,1,0,0,0,NaT
2,2,226,1,0,0,0,NaT
3,3,151,1,0,0,0,NaT
4,4,307,1,0,0,0,NaT
...,...,...,...,...,...,...,...
41183,41183,334,1,0,0,0,NaT
41184,41184,383,1,0,0,0,NaT
41185,41185,189,2,0,0,0,NaT
41186,41186,442,1,0,0,0,NaT


In [121]:
economics

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857
...,...,...,...
41183,41183,94.767,1.028
41184,41184,94.767,1.028
41185,41185,94.767,1.028
41186,41186,94.767,1.028
